# Gaia Least Cost Path Example

This document contains an example of the Gaia Least Cost Path plugin.

The original Python notebook is available in the plugin source code under the docs/examples folder.

In [24]:
# Import required modules
import folium
import numpy as np
from folium.plugins import ImageOverlay
from gaia_leastcostpath.processes import LeastCostProcess
from gaia.geo.geo_inputs import RasterFileIO, FeatureIO

# NOTE:

### This plugin does not currently conform to the standard Gaia framework of using GaiaIO sub-class objects as inputs; this will change in the near future.

Create a LeastCostProcess instance to determine the path between two points across a grid of global precipitation, where lower precipitation == less cost.

In [20]:
# Raster file to be used for least cost values
lcp_raster = RasterFileIO(uri="../../tests/data/globalprecip.tif")

#Start and end points
start_point = FeatureIO(features=[
        {"geometry": {"type": "Point", "coordinates": [-71.590526, 42.659566]}, "properties":{}},
    ])
end_point = FeatureIO(features=[
        {"geometry": {"type": "Point", "coordinates": [-122.817426, 46.561380]}, "properties":{}},
    ])

# Least Cost Path process
lcp = LeastCostProcess(inputs = [
        {
            "uri": lcp_raster.uri,
            "start": start_point.features[0]['geometry']['coordinates'], 
            "end": end_point.features[0]['geometry']['coordinates'], 
        }
    ])

Compute the least cost path

In [32]:
lcp.compute()

Display the path on a map

In [37]:
world_map = folium.Map([50,-90],
                  zoom_start=4,
                  tiles='cartodbpositron')

start_pt = folium.features.GeoJson(start_point.read().to_json(), name='Start')
end_pt = folium.features.GeoJson(end_point.read().to_json(), name='Finish')
temparray = np.array(lcp_raster.read().GetRasterBand(1).ReadAsArray())
overlay = ImageOverlay(temparray, [[-90.0,-180.0],[90.0,180.0]], mercator_project=True, control=True)
overlay.layer_name = 'Least Cost Grid'

least_cost_path = folium.features.GeoJson(lcp.output.data.to_json(), name='Least Cost Path')

world_map.add_children(overlay)
world_map.add_children(least_cost_path)
world_map.add_children(start_pt)
world_map.add_children(end_pt)

folium.LayerControl().add_to(world_map)
world_map